In [75]:
import os
import openai
import sys
sys.path.append('./')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [77]:
get_completion("""
               Formulera en fråga från följande paragraf: 
               Vi använder oss av ett för försäkringsbranschen gemensamt skadeanmälningsregister (GSR). 
               Detta register innehåller vissa uppgifter om skadan samt uppgift om vem som begärt ersättning och används endast i samband med skadereglering. 
               Det innebär att vi får reda på om du tidigare anmält någon skada hos annat försäkringsföretag eller myndighet som handlägger likartade ersättningsanspråk. 
               Ändamålet med GSR är att tillhandahålla ett underlag till försäkringsföretag och myndigheter som handlägger liknande ersättningsanspråk för att identifiera oklara försäkringsfall och ersättningsanspråk. 
               Därigenom kan företag och myndigheter motverka utbetalning av ersättningar som baseras på oriktiga uppgifter.
               Uppgifterna kan även användas i avidentifierad form för statistiska ändamål.
               """)

'Vilket syfte har det gemensamma skadeanmälningsregistret (GSR) inom försäkringsbranschen och hur används det för att identifiera oklara försäkringsfall och ersättningsanspråk?'

## LangChain

### Document loading

In [78]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("data/Fastighet.pdf")
pages = loader.load()

In [79]:
len(pages)

72

In [80]:
# page72 = pages[71].page_content
page72 = pages[71]

In [81]:
page72.metadata

{'source': 'data/Fastighet.pdf', 'page': 71}

In [82]:
page72

Document(page_content='72\nFörsäkringsvillkor\nFastighetsförsäkring\nGrundvillkor C330:FOm uppgifterna överförs till ett land utanför EU, säkerställer vi att sådan överföring är laglig, \nexempelvis genom att använda de standardiserade modellklausuler för dataöverföring som \nantagits av EU-kommissionen och som finns tillgängliga på EU-kommissionens webbplats.\nDu har rätt att få information om vilka uppgifter som vi behandlar om dig, att få felaktiga \nuppgifter rättade, att begära att vi begränsar vår behandling och att dina uppgifter överförs till \nannan part samt invända mot den behandling vi utför. Du har även rätt att inge klagomål till \nansvarig tillsynsmyndighet.\nLäs mer om vår hantering av personuppgifter och dina rättigheter på folksam.se/personuppgifter.\n12. Skaderegistrering\nVi använder oss av ett för försäkringsbranschen gemensamt skadeanmälningsregister (GSR). \nDetta register innehåller vissa uppgifter om skadan samt uppgift om vem som begärt ersättning \noch använd

### Split

In [83]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [84]:
chunk_size = 950
chunk_overlap = 50

In [85]:
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator = "\n"
)
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap, 
    separators=["\n"]
    # separators=["\\n", " \n", " ", ""]
)

In [86]:
c_split = c_splitter.split_documents(pages)
len(c_split)
c_split[279:]

[Document(page_content='72\nFörsäkringsvillkor\nFastighetsförsäkring\nGrundvillkor C330:FOm uppgifterna överförs till ett land utanför EU, säkerställer vi att sådan överföring är laglig, \nexempelvis genom att använda de standardiserade modellklausuler för dataöverföring som \nantagits av EU-kommissionen och som finns tillgängliga på EU-kommissionens webbplats.\nDu har rätt att få information om vilka uppgifter som vi behandlar om dig, att få felaktiga \nuppgifter rättade, att begära att vi begränsar vår behandling och att dina uppgifter överförs till \nannan part samt invända mot den behandling vi utför. Du har även rätt att inge klagomål till \nansvarig tillsynsmyndighet.\nLäs mer om vår hantering av personuppgifter och dina rättigheter på folksam.se/personuppgifter.\n12. Skaderegistrering\nVi använder oss av ett för försäkringsbranschen gemensamt skadeanmälningsregister (GSR).', metadata={'source': 'data/Fastighet.pdf', 'page': 71}),
 Document(page_content='Detta register innehåller

In [87]:
r_split = r_splitter.split_documents(pages)
len(r_split)
r_split[279:]

[Document(page_content='72\nFörsäkringsvillkor\nFastighetsförsäkring\nGrundvillkor C330:FOm uppgifterna överförs till ett land utanför EU, säkerställer vi att sådan överföring är laglig, \nexempelvis genom att använda de standardiserade modellklausuler för dataöverföring som \nantagits av EU-kommissionen och som finns tillgängliga på EU-kommissionens webbplats.\nDu har rätt att få information om vilka uppgifter som vi behandlar om dig, att få felaktiga \nuppgifter rättade, att begära att vi begränsar vår behandling och att dina uppgifter överförs till \nannan part samt invända mot den behandling vi utför. Du har även rätt att inge klagomål till \nansvarig tillsynsmyndighet.\nLäs mer om vår hantering av personuppgifter och dina rättigheter på folksam.se/personuppgifter.\n12. Skaderegistrering\nVi använder oss av ett för försäkringsbranschen gemensamt skadeanmälningsregister (GSR).', metadata={'source': 'data/Fastighet.pdf', 'page': 71}),
 Document(page_content='Detta register innehåller

### Embedding

In [89]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

persist_directory = "data/chroma"
embedding = OpenAIEmbeddings()

In [90]:
!rm -rf ./docs/chroma  # remove old database files if any

In [93]:
vectordb = Chroma.from_documents(
    documents=c_split,
    embedding=embedding,
    persist_directory=persist_directory
)

print(vectordb._collection.count)

<bound method Collection.count of Collection(name=langchain)>


In [116]:
question1 = 'Vilket syfte har det gemensamma skadeanmälningsregistret inom försäkringsbranschen?'
question2 = 'Vad är undantaget för försäkring av rån?'

In [117]:
docs = vectordb.similarity_search(question1, k=3)
len(docs)

3

In [118]:
print(docs[0].page_content)

Detta register innehåller vissa uppgifter om skadan samt uppgift om vem som begärt ersättning 
och används endast i samband med skadereglering. Det innebär att vi får reda på om du tidigare 
anmält någon skada hos annat försäkringsföretag eller myndighet som handlägger likartade 
ersättningsanspråk. Ändamålet med GSR är att tillhandahålla ett underlag till försäkringsfö -
retag och myndigheter som handlägger liknande ersättningsanspråk för att identifiera oklara 
försäkringsfall och ersättningsanspråk. Därigenom kan företag och myndigheter motverka 
utbetalning av ersättningar som baseras på oriktiga uppgifter. Uppgifterna kan även användas 
i avidentifierad form för statistiska ändamål.
Personuppgiftsansvarig för GSR är Skadeanmälningsregister (GSR) AB, Box 24171, 104 51 
Stockholm. Se www.gsr.se för mer information om den behandling av uppgifter som förekommer 
i registret.


In [119]:
print(docs[1].page_content)

i registret.
Vi kan också lämna uppgifter om bland annat stölder och eftersökt gods till Larmtjänst AB, en 
branschgemensam organisation som arbetar för att bekämpa försäkringsrelaterad brottslighet. 
13. Försäkringsavtalslagen med mera
För denna försäkring gäller i övrigt bestämmelserna i försäkringsavtalslagen (2005:104) och 
övrig svensk rätt.


In [120]:
print(docs[2].page_content)

72
Försäkringsvillkor
Fastighetsförsäkring
Grundvillkor C330:FOm uppgifterna överförs till ett land utanför EU, säkerställer vi att sådan överföring är laglig, 
exempelvis genom att använda de standardiserade modellklausuler för dataöverföring som 
antagits av EU-kommissionen och som finns tillgängliga på EU-kommissionens webbplats.
Du har rätt att få information om vilka uppgifter som vi behandlar om dig, att få felaktiga 
uppgifter rättade, att begära att vi begränsar vår behandling och att dina uppgifter överförs till 
annan part samt invända mot den behandling vi utför. Du har även rätt att inge klagomål till 
ansvarig tillsynsmyndighet.
Läs mer om vår hantering av personuppgifter och dina rättigheter på folksam.se/personuppgifter.
12. Skaderegistrering
Vi använder oss av ett för försäkringsbranschen gemensamt skadeanmälningsregister (GSR).


In [121]:
# Persist Chroma VectorDB
vectordb.persist()

### Retrieve

In [129]:
# Retrieval methods -> MMR to enfore diversity in the search result
q2_mmr = vectordb.max_marginal_relevance_search(question2, k=3, fetch_k=5)
len(q2_mmr)

# q2_similiarity = vectordb.similarity_search(question2, k=3)
# len(q2_similiarity)

3

In [130]:
# TODO
# Try different retrievers
# TF-IDF
# SVM
# SelfQuery
# ContextualCompression

### Chat

In [133]:
question = "Vilka skadehändelser täcker egendomsförsäkringen?"

In [134]:
vectordb._collection.count()

282

In [135]:
docs = vectordb.similarity_search(question, k=3)
len(docs)

3

In [137]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

In [138]:
from langchain.chains import RetrievalQA

In [139]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [141]:
result = qa_chain({"query": question})
print(result['result'])

Egendomsförsäkringen täcker skadehändelser som inträffar i samband med egendomsskada, såsom skador på försäkrad egendom vid räddningsåtgärder, förlust eller stöld av egendom. Den täcker också skador orsakade av den försäkrade och dess arbetstagare i tjänsten. Vidare täcker försäkringen ansvar för fastighet om den försäkrade är ägare till fastigheten eller brukare av fastigheten och har övertagit fastighetsägaransvaret enligt avtal. Byggnads- och anläggningsarbeten på fastigheten omfattas också av försäkringen, om kostnaderna för detta inte överstiger 20 basbelopp.


### Prompt

In [142]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Använd följande delar av sammanhanget för att svara på frågan i slutet. Om du inte vet svaret, säg bara att du inte vet, försök inte hitta på ett svar. Använd maximalt tre meningar. Håll svaret så kortfattat och enkelt att förstå. 
{context}
Fråga: {question}
Svar:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [143]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [154]:
question = "Hur mycket är självrisk för besöksolycksfall?"
result = qa_chain({"query": question})
print(result['result'])

Det framgår inte av det givna sammanhanget vad självrisken är för besöksolycksfall.


In [155]:
result["source_documents"][0]

Document(page_content='10.3.4 Särskild självrisk  – Skada genom tätskikt\nSjälvrisken är 0,5 basbelopp utöver annars gällande självrisk vid skada på byggnad genom \nläckage från\n• golv- eller väggtätskikt eller vid dess anslutning till installationer eller övriga byggnadsdelar\n10.3.5 Särskild självrisk – Skada på grund av åldersförändring\nSjälvrisken är 1 basbelopp utöver annars gällande självrisk vid skada på byggnad genom \nläckage från\n• installationer, om läckaget orsakats av åldersförändringar (till exempel korrosion eller för -\nslitning) och installationens ålder är mindre än 50 år. För installation som är 50 år eller äldre \ngäller istället en utökad självrisk med 3 basbelopp utöver annars gällande självrisk.\n10.3.6 Särskild självrisk – Skada genom frysning \nSjälvrisken är ett basbelopp utöver annars gällande självrisk, vid skada på byggnad genom \nfrysning. \n10.4 Inbrottsförsäkring \nInbrottsförsäkringen omfattar skadehändelser enligt 10.4.2 – 10.4.6 nedan.', metadata={

In [156]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain_mr({"query": question})
print(result['result'])

Det finns ingen information om självrisk för besöksolycksfall i den givna texten.


### Memory

In [157]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [158]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [159]:
question = "Vad handlar det dokumentet om?"
result = qa({"question": question})

print(result['answer'])

Dokumentet handlar om försäkringsvillkor för fastighetsförsäkring. Det innehåller information om olika typer av försäkringsvillkor, vad som omfattas av försäkringen, självrisk, skattemål och annan relevant information.


In [160]:
question = "Är hyresgäst täckes av denna fastighetsförsäkring?"
result = qa({"question": question})

print(result['answer'])

Ja, hyresgästen täcks av denna fastighetsförsäkring om de är boende i den försäkrade fastigheten och har en giltig hemförsäkring.


In [161]:
question = "Vad är självrisken för hemförsäkring?"
result = qa({"question": question})

print(result['answer'])

Jag vet inte.


In [162]:
question = "Vilken företag handlar detta dokument om?"
result = qa({"question": question})

print(result['answer'])

Dokumentet handlar om försäkringsbolaget Folksam Ömsesidig Sakförsäkring.


In [163]:
question = "Ingår rättsskydd i denna försäkring?"
result = qa({"question": question})

print(result['answer'])

Det framgår inte tydligt om rättsskydd ingår i denna försäkring. Du bör kontrollera försäkringsbrevet eller kontakta försäkringsbolaget för att få mer information om detta.


In [164]:
result

{'question': 'Ingår rättsskydd i denna försäkring?',
 'chat_history': [HumanMessage(content='Vad handlar det dokumentet om?', additional_kwargs={}, example=False),
  AIMessage(content='Dokumentet handlar om försäkringsvillkor för fastighetsförsäkring. Det innehåller information om olika typer av försäkringsvillkor, vad som omfattas av försäkringen, självrisk, skattemål och annan relevant information.', additional_kwargs={}, example=False),
  HumanMessage(content='Är hyresgäst täckes av denna fastighetsförsäkring?', additional_kwargs={}, example=False),
  AIMessage(content='Ja, hyresgästen täcks av denna fastighetsförsäkring om de är boende i den försäkrade fastigheten och har en giltig hemförsäkring.', additional_kwargs={}, example=False),
  HumanMessage(content='Vad är självrisken för hemförsäkring?', additional_kwargs={}, example=False),
  AIMessage(content='Jag vet inte.', additional_kwargs={}, example=False),
  HumanMessage(content='Vilken företag handlar detta dokument om?', addit